# Example procedure to estimate LAI from satellite weather data
---
**Authors**
- [Mattia C. Mancini](https://github.com/mcmancini) -- (m.c.mancini@exeter.ac.uk)
- [Mingyuan Chen](https://github.com/MingyuanChen94) -- (M.Chen2@exeter.ac.uk)  

**Date**: May 23rd, 2024  

---

This notebook goes through the process of estimating Leaf Area Index of crops following the methodology and using the code developed by [Myrgiotis *et al.*, 2021](https://datashare.ed.ac.uk/handle/10283/4086). The code has been taken from the [LAI_S1S2](https://datashare.ed.ac.uk/bitstream/handle/10283/4086/LAI_S1S2_fusion.py?sequence=2&isAllowed=y) script, and heavily modified to make it work. This is now in the `agromanagment/utiliy/lai_generator.py` module. 
For this to work, the following steps are required:
- 1. Download and install the [ESA SNAP application](https://step.esa.int/main/download/snap-download/): select "All Toolboxes", although the Sentinel Toolboxes should be enough;
- 2. Install into your global environment the GDAL library, if not yet vailable. Note: this is a global installation, not the installation already exisiting in the `agromanagment` Conda environment. To install GDAL globally in Windodws:
    - Download [OSGeo4W](https://trac.osgeo.org/osgeo4w/).
    - Once downloaded, add to your PATH the `bin` folder in the main OSGeo4W installation folder, for example `C:\OSGeo4W\bin` if you have used the default installation location. This allows to access from terminal all the various gdal scripts required to perform spatial analyisis.  

Once completed the previous steps, we can then estimate the LAI through the gap-filling algorithm. This assumes that the weather data of interest has been already downloaded. This is done using the example notebooks 2 (Sentinel 1 and 2 data) and 3 (ERA 5 reanalysis data).

## Computing LAI
Computing LAI consists of the following steps:
1. Initialise an instance of the class `LaiGenerator`
2. Call the method `s1_to_vvvh()` which processes Sentinel 1 data into backscatter VV/VH intensity maps for the selected area;
3. Call the method `s2_to_lai()` which processeses Sentinel-2 images/tiles into Leaf Area Index for the entire S2 tile;
4. Call the method `lai_ts_creation()` which collects the outputs previous two methods, and creates time-series of vapour pressure deficit (VPD) using data downloaded from ECMWF ERA-5. It then applies the gap-filling algorithm to generate weekly continuous LAI time-series for the location and timeframe of interest.


In [ ]:
# Update sys path so notebook can access the agromanagement package
import sys
sys.path.append('../')

### 1.1. Temporal timeframe

In [ ]:
START_DATE = "2019-01-01"
END_DATE = "2019-12-31"

### 1.2. Paths for data

In [ ]:
import os

# replace paths for user-defined ones
ROOT_DIR = os.path.join("..")
SENTINEL_1_OUTPUT_FOLDER = os.path.join("..", "resources", "sentinel_1")
SENTINEL_2_OUTPUT_FOLDER = os.path.join("..", "resources", "sentinel_l2a")
ERA_OUTPUT_FOLDER = os.path.join("..", "resources", "era_5")
PARCEL_FOLDER = os.path.join("..", "resources")
HOME_FOLDER = os.path.expanduser("~")
SNAP_GRAPH_DIR = f"{HOME_FOLDER}/.snap/graphs/"
SNAP_GTP_DIR = "C:/Program Files/esa-snap/bin/"


### 1.3. Define parcel of interest

In [ ]:
import geopandas as gpd

PARCEL_NAME = "1270998.geojson"
parcel_path = os.path.join(PARCEL_FOLDER, PARCEL_NAME)

### 1.3. Initialise LAI Generator

In [ ]:
from agromanagement.utility.lai_generator import LaiGenerator

filename = ''.join([char for char in PARCEL_NAME if char.isdigit()])

lai = LaiGenerator(
    jsonloc=parcel_path,
    filename=filename,
    workingdir=ROOT_DIR,
    s1_dir=SENTINEL_1_OUTPUT_FOLDER,
    s2_dir=SENTINEL_2_OUTPUT_FOLDER,
    era_dir=ERA_OUTPUT_FOLDER,
    snap_graphs_dir=SNAP_GRAPH_DIR,
    snap_gtp_dir=SNAP_GTP_DIR,
    startdate=START_DATE,
    enddate=END_DATE,
)

### 1.4. Compute LAI
This takes a long time, and will return plenty of warnings which for the time being we do not care about: I tried to mute them, but partly unsuccessfully. These warnings Will need to be investigated and addressed at some point, what I wanted was to have a script working and returning LAI values without returning errors. Some of the warnings have FutureWarnings from Pandas on the way it is dealing with concatenating together dataframes with NAs; others are returned by snap and seem more concerning. Interestingly though, these snap-related warnings are returned only when running the snap graphs from Python but not when running them using the SNAP GUI. 

In [ ]:
lai.s1_to_vvvh()
lai.s2_to_lai()
lai.lai_ts_creation()